In [2]:
import matplotlib

In [3]:
matplotlib.use('Qt5Agg')

In [4]:
import numpy as np

In [5]:
import pandas as pd

In [6]:
import matplotlib.pyplot as plt

In [7]:
import mne

In [11]:
data = mne.io.read_raw_edf('./P1108 resting mix.bdf', preload = True, montage = 'biosemi64')

Extracting EDF parameters from ./P1101.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
The following EEG sensors did not have a position specified in the selected montage: ['EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8', 'GSR1', 'GSR2', 'Erg1', 'Erg2', 'Resp', 'Plet', 'Temp']. Their position has been left untouched.
Reading 0 ... 5900287  =      0.000 ...  2881.000 secs...


<ipython-input-11-8652aed6f4e4>:1: RuntimeWarning: The following EEG sensors did not have a position specified in the selected montage: ['EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8', 'GSR1', 'GSR2', 'Erg1', 'Erg2', 'Resp', 'Plet', 'Temp']. Their position has been left untouched.
  data = mne.io.read_raw_edf('./P1101.bdf', preload = True, montage = 'biosemi64')


In [12]:
x = mne.find_events(data)

Trigger channel has a non-zero initial value of 130816 (consider using initial_event=True to detect this event)
240 events found
Event IDs: [65301 65302 65303 65304 65305 65332 65351 65352 65361 65381 65382 65383
 65384 65505]


In [ ]:
x[:, 2] -= x[:, 1]
x[:, 1] -= x[:, 1]
x

In [ ]:
#data.crop(tmin=x[1,0]/2048, tmax=x[2,0]/2048)
data.crop(tmin=2, tmax=590)

In [10]:
data.plot();

In [ ]:
data.info

In [ ]:
print(data.ch_names)

In [ ]:
data.plot_psd();

In [11]:
data.set_eeg_reference();

Applying average reference.
Applying a custom EEG reference.


In [ ]:
#data.set_eeg_reference(ref_channels=['Fz'], projection = False);

In [ ]:
from mne.preprocessing import ICA

In [ ]:
data_to_ica = data.copy()
data_to_ica.filter(1, None)
data_to_ica.filter(None, 40)
data_to_ica.resample(128) #theorem Nquista, ustaw 2 * taki sample jaki chcesz wyjąć z danych (jeśli max w sygnale to 200, wybierz 400)

In [ ]:
#data.filter(1,30,fir_design='firwin') # drastyczne cięcie
#data_to_ica.filter(0.1,40,fir_design='firwin') #rekomendowane cięcie
data_to_ica.notch_filter(50)

In [ ]:
ica_data = ICA(method = 'picard', random_state = 0, max_iter = 500)
ica_data.fit(data_to_ica)

In [ ]:
ica_data.plot_components(range(20), inst = data_to_ica)

In [ ]:
ica_data.plot_properties(data_to_ica)

In [ ]:
ica_data.get_sources(data_to_ica).plot()

In [ ]:
ica_data.exclude = [0,2] # numery komponentów do wywalenia
ica_data.apply(data)

In [ ]:
data.plot_psd();

In [ ]:
data.plot();

In [ ]:
data.save('czyste_eeg_raw.fif')